Following Python jupyter notebook presents the workflow for calculation of Earthquake Count and
Total Seismic moment release summarized within 20x20km grid cells as described in Hypothesis
Testing Section of the Chapter 4.

It takes following inputs: 
1. mont_grid = previously calculated polygon grid in in which each row represents the grid cell 
2. file_obs = observed seismicity catalog 
3. filesTests = folder with calculated synthetic earthquake catalogs

It can be run in Jupyter notebook or Jupyter Lab (https://jupyter.org/).

In [32]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely import wkt
from csv import writer
import re

In [33]:
""" Calculate Seismic Moment from Moment Magnitude"""
def calc_seismic_moment(momentMag):
    seismMoment = pow(10, (3/2 * momentMag+16.0))
    seismMoment = seismMoment / 10000000 # to convert to Newton-meter
    return seismMoment

"""calculate EQs within R km buffer from the grid cell cetroid"""
def count_eq(cell, eqs):
    n_eqs = eqs.within(cell).sum()
    return n_eqs

"""calculate cumulative seismic moment per grid cell """
def count_seism_moment(cell, eqs):
        cum_M0 = eqs[eqs.within(cell)]['M0Synt'].sum()
        return cum_M0

"""append count results to file"""
def append_results(results, file):
    csv_input = pd.read_csv(file)
    old_last_col = csv_input.columns[-1]
    new_last_col = 'sim_' + str(int(re.findall('\d+', old_last_col)[0]) + 1)
    csv_input.loc[:,new_last_col] = results.values
    csv_input.to_csv(file, index=False)
    """append correlation results to file"""
    
def append_results_to_list(results, file):
    csv_input = pd.read_csv(file)
    old_last_col = csv_input.columns[-1]
    new_last_col = 'sim_' + str(int(re.findall('\d+', old_last_col)[0]) + 1)
    csv_input.loc[:,new_last_col] = results.values
    csv_input.to_csv(file, index=False)


In [34]:
### 1st iteration

In [35]:
# Load Montney grid
mont_grid = gpd.read_file("data/mont_grid/montey20x20grid.shp")
mont_gridEQ = mont_grid.to_crs(26911)
mont_gridM0 = mont_grid.to_crs(26911)

mont_gridEQ_obs = mont_grid.to_crs(26911)
mont_gridM0_obs = mont_grid.to_crs(26911)

In [36]:
file_obs = 'data/compiled_eq_NRCanAER_KSMMA_NPGMMA_AB.csv'
EQcat_obs = pd.read_csv(file_obs)
EQcat_obs = EQcat_obs[EQcat_obs.mag>=2]
EQcat_obs.loc[:,'M0Synt']= EQcat_obs.apply(lambda x:calc_seismic_moment(x['mag']), axis=1)

EQcat_obs['geometry'] = EQcat_obs['geometry'].apply(wkt.loads)
EQcat_obs_gdf = gpd.GeoDataFrame(EQcat_obs, geometry=EQcat_obs.geometry,crs="EPSG:4326")
EQcat_obs_gdf = EQcat_obs_gdf.to_crs(crs="EPSG:26911")

### Count and assign the Earthquake COunt and Total Seismic Moment to each gridcell
EQCount_obs = mont_gridEQ_obs.apply(lambda x: count_eq(x.geometry,EQcat_obs_gdf), axis=1)

M0Count_obs = mont_gridM0_obs.apply(lambda x: count_seism_moment(x.geometry, EQcat_obs_gdf), axis=1)
EQObsvec = EQCount_obs - np.mean(EQCount_obs)
M0Obsvec = M0Count_obs - np.mean(M0Count_obs)

C:\Users\Paulina-laptop\AppData\Roaming\Python\Python37\site-packages\geopandas\array.py:166: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.from_shapely(data), crs=crs)


In [37]:
import glob
filesTests = glob.glob('data/SimulatedCatalogues/A/trimmed_catalogs/*')

In [38]:
file0 = filesTests[0]
EQcat = pd.read_csv(file0)
EQcat['geometry'] = EQcat['geometry'].apply(wkt.loads)
EQcat_gdf = gpd.GeoDataFrame(EQcat, geometry=EQcat.geometry, crs="EPSG:26910")
EQcat_gdf = EQcat_gdf.to_crs(crs="EPSG:26911")

mont_gridEQ.loc[:,'sim_0'] = mont_gridEQ.apply(lambda x: count_eq(x.geometry, EQcat_gdf), axis=1)
mont_gridM0.loc[:,'sim_0'] = mont_gridM0.apply(lambda x: count_seism_moment(x.geometry, EQcat_gdf), axis=1)
mont_gridEQ.to_csv('results/EQCountResults/EQCount_testA.csv')
mont_gridM0.to_csv('results/M0CountResults/M0Count_testA.csv')

EQCount = mont_gridEQ.loc[:,'sim_0']
M0Count = mont_gridM0.loc[:,'sim_0']

EQSynvec = EQCount - np.mean(EQCount)
M0Synvec = M0Count - np.mean(M0Count)

phiSynEQ = np.sum(EQObsvec*EQSynvec) /np.sqrt(np.sum(EQObsvec**2) * np.sum(EQSynvec**2))
phiSynM0 = np.sum(M0Obsvec*M0Synvec) /np.sqrt(np.sum(M0Obsvec**2) * np.sum(M0Synvec**2))

d = {'CorrCoeff_EQ': [phiSynEQ], 'CorrCoeff_M0': [phiSynM0]}

df = pd.DataFrame(data=d)
df.to_csv('results/corrResultsA.csv')

In [39]:
### Next iterations

In [40]:
path = 'data/SimulatedCatalogues/A/trimmed_catalogs/*'
file_list = [file for file in glob.glob(path)]

for file_nr, file in enumerate(file_list[1:]):  

    try:
        EQcat = pd.read_csv(file)
        EQcat['geometry'] = EQcat['geometry'].apply(wkt.loads)
        EQcat_gdf = gpd.GeoDataFrame(EQcat, geometry=EQcat.geometry, crs="EPSG:26910")
        EQcat_gdf = EQcat_gdf.to_crs(crs="EPSG:26911")
    except:
        pass

    EQCount = mont_gridEQ.apply(lambda x: count_eq(x.geometry, EQcat_gdf), axis=1) 
    M0Count = mont_gridM0.apply(lambda x: count_seism_moment(x.geometry, EQcat_gdf), axis=1)
    
    # save grids to file

    append_results(EQCount, 'results/EQCountResults/EQCount_testA.csv')
    append_results(M0Count, 'results/M0CountResults/M0Count_testA.csv')
    
    EQSynvec = EQCount - np.mean(EQCount)
    M0Synvec = M0Count - np.mean(M0Count)

    phiSynEQ = np.sum(EQObsvec*EQSynvec) /np.sqrt(np.sum(EQObsvec**2) * np.sum(EQSynvec**2))
    phiSynM0 = np.sum(M0Obsvec*M0Synvec) /np.sqrt(np.sum(M0Obsvec**2) * np.sum(M0Synvec**2))
    
    List = [np.int(file_nr)+1, phiSynEQ, phiSynM0]
    
    # save to correlation results to file
    with open('results/corrResultsA.csv', 'a', newline='') as f_object:

        writer_object = writer(f_object)
        writer_object.writerow(List)
        f_object.close()

D:\Programs\Anaconda3\envs\geological_susceptibility\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
